In [3]:
from CatanEnv import CatanSetupEnv
from sb3_contrib.ppo_mask import MaskablePPO

env = CatanSetupEnv()

model = MaskablePPO("MlpPolicy", env, gamma=0.4, verbose=1) #, tensorboard_log="./tensorboard_logs/")
# model.learn(total_timesteps=50_000) #, tb_log_name='simple_setup_phase_training')

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [7]:
from sb3_contrib.common.maskable.utils import get_action_masks
from GameStateViewer import SaveGameStateImage
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from ModelState import getInputState

##############################################################
rewardList = []
rewardList100 = []
winner = [0,0,0,0]

for episode in range(1):
    done = False
    state, info = env.reset()

    while done != True:
        action_masks = get_action_masks(env)
        action, _states = model.predict(state, action_masks=action_masks)
        state, reward, done, _, info = env.step(action.item())
        # print(f"Reward: {reward+7}")
        rewardList.append(reward)
    
    # Once initial placement is done use random agents till game is over
    while True:
        currPlayer = env.game.gameState.players[env.game.gameState.currPlayer]

        agentAction = currPlayer.DoMove(env.game)
        agentAction.ApplyAction(env.game.gameState)
        print(len(getInputState(env.game.gameState)))

        if env.game.gameState.currState == "OVER":
            break
    winner[env.game.gameState.winner] += 1


print(winner)



# plt.plot(rewardList)
# plt.xlabel('Episode (100)')
# plt.ylabel('Avg VP')
# plt.show()

# img = mpimg.imread('TRAINING.png')
# plt.imshow(img)
# plt.axis('off')  # Optional: Remove axes for a clean display
# plt.show()

In [ ]:
###################################################################################################################################################

In [6]:
from CatanEnv import CatanSetupEnv, CatanEnv
from sb3_contrib.ppo_mask import MaskablePPO

env = CatanEnv()

model = MaskablePPO("MlpPolicy", env, gamma=0.4, verbose=1, tensorboard_log="./tensorboard_logs/")
model.learn(total_timesteps=500_000, tb_log_name='full_training')

In [5]:
from sb3_contrib.common.maskable.utils import get_action_masks
from GameStateViewer import SaveGameStateImage
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from ModelState import getInputState

##############################################################
rewardList = []
rewardList100 = []
winner = [0,0,0,0]
vpList = []

for episode in range(100):
    done = False
    state, info = env.reset()

    while done != True:
        action_masks = get_action_masks(env)
        action, _states = model.predict(state, action_masks=action_masks)
        state, reward, done, _, info = env.step(action.item())
        # print(f"Reward: {reward+7}")
        rewardList.append(reward)
    
    winner[env.game.gameState.winner] += 1
    vpList.append(env.players[2].victoryPoints)


print(winner)
print(sum(vpList)/len(vpList))

[27, 26, 15, 32]
5.37


In [ ]:
###################################################################################################################################################

In [5]:
from DeepLearning.Env import NoSetupEnv
from sb3_contrib.ppo_mask import MaskablePPO
import os
os.environ['SAVE_IMAGE'] = 'False'

env = NoSetupEnv(setupModel=MaskablePPO.load('DeepLearning/Models/SetupRandom_wins_100k.zip'))

model = MaskablePPO("MlpPolicy", env, gamma=0.4, verbose=1, tensorboard_log="./tensorboard_logs/")
model.learn(total_timesteps=1_000_000, tb_log_name='ENV-NoSetup_REWARD-win')

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./tensorboard_logs/ENV-NoSetup_REWARD-win_1


/Users/danieldrummond/Catan/PyCatron/TC2/Client/env/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.action_masks to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.action_masks` for environment variables or `env.get_wrapper_attr('action_masks')` that will search the reminding wrappers.
  logger.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 193      |
|    ep_rew_mean     | -0.5     |
| time/              |          |
|    fps             | 1748     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 224         |
|    ep_rew_mean          | -0.176      |
| time/                   |             |
|    fps                  | 1191        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009324359 |
|    clip_fraction        | 0.127       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.876      |
|    explained_variance   | -0.98       |
|    learning_rate        | 0.

In [8]:
from sb3_contrib.common.maskable.utils import get_action_masks
from DeepLearning.Env import NoSetupEnv
import os
from Game.CatanPlayer import PlayerStatsTracker
from sb3_contrib.ppo_mask import MaskablePPO

# os.environ['SAVE_IMAGE'] = 'True'
env = NoSetupEnv(setupModel=MaskablePPO.load('DeepLearning/Models/SetupRandom_wins_100k.zip'))
model = MaskablePPO.load('DeepLearning/Models/NoSetup_wins_1M.zip')

rewardList = []
rewardList100 = []
winner = [0,0,0,0]
vpList = []

stats = PlayerStatsTracker()
randomStats = PlayerStatsTracker()

for episode in range(1000):
    done = False
    state, info = env.reset()

    while done != True:
        action_masks = get_action_masks(env)
        action, _states = model.predict(state, action_masks=action_masks)
        state, reward, done, _, info = env.step(action.item())
        # print(f"Reward: {reward+7}")
        rewardList.append(reward)

    
    winner[env.game.gameState.winner] += 1
    
    env.game.gameState.players[0].generatePlayerStats()
    env.game.gameState.players[1].generatePlayerStats()
    # print(env.game.gameState.players[0].stats)
    stats += env.game.gameState.players[0].stats
    randomStats += env.game.gameState.players[1].stats

stats.getAverages()
randomStats.getAverages()
print(stats)
print("\n***********************\n")
print(randomStats)

print("Winnings: ", winner)

General
 numTurns: 59.887
 victoryPoints: 9.295
 finalTradeRates: [3.582, 3.596, 3.606, 3.584, 3.582]
Dev Card Breakdown
 devCardsBought: 5.527
 usedDevCards: [3.075, 0.301, 0.38, 0.403, 0.0]
Point Breakdown
 settlementsBuilt: 1.925
 citiesBuilt: 1.925
 devCardVP: 1.176
 largestArmy: 0.544
 longestRoad: 0.615
Resource Breakdown
 resourcesReceived: [36.762, 43.278, 27.391, 50.503, 38.842]
 totalResourcesDiscarded: 33.183
 totalResourcesStolen: 10.928
 resourcesFromDevCard: [0.585, 0.777, 0.252, 0.267, 0.011]
 resourcesFromBankTrade: [7.129, 10.586, 4.441, 7.393, 9.17]
 finalResourceProduction: [8.428, 9.966, 7.029, 11.79, 9.547]
Setup Breakdown
 setupResourceProduction: [4.034, 4.269, 2.853, 4.893, 4.326]
 setupTradeRates: [3.988, 3.992, 3.99, 3.988, 3.982]


***********************

General
 numTurns: 59.379
 victoryPoints: 4.579
 finalTradeRates: [3.433, 3.473, 3.463, 3.43, 3.406]
Dev Card Breakdown
 devCardsBought: 3.542
 usedDevCards: [1.932, 0.236, 0.282, 0.282, 0.0]
Point Breakdow